In [1]:
import os
import sys
import numpy as np
import wandb

from playlist_recommender.modelling import model_pipeline
from playlist_recommender.modelling import utils
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn import metrics
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder

In [2]:
X, y = utils.prep_playlist_df()
X_train, X_test, y_train, y_test = model_pipeline.make_best_transformation_pipeline(
    X, y
)
X_train.shape, X_test.shape, y_train.shape, y_test.shape,

((7293, 1052), (3927, 1052), (7293,), (3927,))

In [3]:
le = LabelEncoder()
le.fit(y_train)

LabelEncoder()

In [4]:
# XGBoost requires labelled targets

y_train = le.transform(y_train)
y_test = le.transform(y_test)

# Sweeps

In [5]:
sweep_config = {
    "method": "bayes",  # try grid or random
    "metric": {"name": "f1_score", "goal": "maximize"},
    "parameters": {
        "booster": {"values": ["gbtree", "gblinear"]},
        "max_depth": {"values": [3, 6, 9, 12]},
        "learning_rate": {"values": [0.1, 0.05, 0.2]},
        "subsample": {"values": [1, 0.5, 0.3]},
    },
}

In [6]:
sweep_id = wandb.sweep(sweep_config, project="spotify-recommender")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Create sweep with ID: 9vz4uz2e
Sweep URL: https://wandb.ai/jmoro/spotify-recommender/sweeps/9vz4uz2e


In [7]:
def train():
    config_defaults = {
        "booster": "gbtree",
        "max_depth": 3,
        "learning_rate": 0.1,
        "subsample": 1,
    }
    with wandb.init(
        project="spotify-recommender",
        tags=["xgboost"],
        name="XGBoost regression tuning",
        config=config_defaults,
    ):
        config = wandb.config
        # fit model on train
        model = XGBClassifier(
            booster=config.booster,
            max_depth=config.max_depth,
            learning_rate=config.learning_rate,
            subsample=config.subsample,
        )
        model.fit(X_train, y_train)

        # make predictions on test
        y_pred = model.predict(X_test)

        # evaluate predictions
        f1_score = metrics.f1_score(y_test, y_pred, average="macro", zero_division=0)
        accuracy = metrics.accuracy_score(y_test, y_pred)
        precision = metrics.precision_score(
            y_test, y_pred, average="macro", zero_division=0
        )
        recall = metrics.recall_score(y_test, y_pred, average="macro")
        wandb.log(
            {
                "f1_score": f1_score,
                "accuracy": accuracy,
                "precision": precision,
                "recall": recall,
            }
        )

In [8]:
wandb.agent(sweep_id, train)

wandb: Agent Starting Run: lfxz44j2 with config:
wandb: 	booster: gblinear
wandb: 	learning_rate: 0.05
wandb: 	max_depth: 12
wandb: 	subsample: 0.5
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: jmoro. Use `wandb login --relogin` to force relogin
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


[14:19:52] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-37/xgboost/src/learner.cc:627: 
Parameters: { "max_depth", "subsample" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




accuracy,▁
f1_score,▁
precision,▁
recall,▁
accuracy,0.81665
f1_score,0.80537
precision,0.81534
recall,0.81665


wandb: Agent Starting Run: 70pftqhc with config:
wandb: 	booster: gbtree
wandb: 	learning_rate: 0.2
wandb: 	max_depth: 9
wandb: 	subsample: 1
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


accuracy,▁
f1_score,▁
precision,▁
recall,▁
accuracy,0.87675
f1_score,0.86771
precision,0.86507
recall,0.87675


...

wandb: Agent Starting Run: xqt3skxs with config:
wandb: 	booster: gbtree
wandb: 	learning_rate: 0.2
wandb: 	max_depth: 9
wandb: 	subsample: 0.5
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


accuracy,▁
f1_score,▁
precision,▁
recall,▁
accuracy,0.87828
f1_score,0.86958
precision,0.86778
recall,0.87828


wandb: Agent Starting Run: 4op1nfqi with config:
wandb: 	booster: gbtree
wandb: 	learning_rate: 0.2
wandb: 	max_depth: 9
wandb: 	subsample: 0.5
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


accuracy,▁
f1_score,▁
precision,▁
recall,▁
accuracy,0.87828
f1_score,0.86958
precision,0.86778
recall,0.87828


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.
